# Round Robin Task Scheduler

In [ ]:
class Task:
    def __init__(self, task_id, name, burst_time, priority=1):
        self.task_id = task_id
        self.name = name
        self.burst_time = burst_time  # CPU time needed
        self.remaining_time = burst_time
        self.priority = priority
        self.waiting_time = 0
        self.turnaround_time = 0
    
    def __str__(self):
        return f"{self.task_id}: {self.name} (Time: {self.burst_time}, Priority: {self.priority})"
    
    def execute(self, time_quantum):
        """Execute task for given time quantum"""
        if self.remaining_time <= time_quantum:
            executed = self.remaining_time
            self.remaining_time = 0
        else:
            executed = time_quantum
            self.remaining_time -= time_quantum
        
        return executed, self.remaining_time == 0


class RoundRobinScheduler:
    def __init__(self, time_quantum=4):
        self.tasks = DoublyCircularLinkedList()  # Using doubly circular for easy navigation
        self.time_quantum = time_quantum
        self.current_time = 0
        self.completed_tasks = []
        self.task_counter = 1
    
    def add_task(self, name, burst_time, priority=1):
        """Add a new task to the scheduler"""
        task = Task(self.task_counter, name, burst_time, priority)
        self.tasks.insert_at_end(task)
        self.task_counter += 1
        print(f"Added task: {task}")
    
    def remove_task(self, task_id):
        """Remove a task by ID"""
        if self.tasks.head is None:
            return False
        
        current = self.tasks.head
        start = current
        
        while True:
            if current.data.task_id == task_id:
                # Found the task
                task = current.data
                
                if current == self.tasks.head:
                    self.tasks.delete_from_beginning()
                elif current == self.tasks.head.prev:  # Last node
                    self.tasks.delete_from_end()
                else:
                    # Remove from middle (simplified)
                    prev = current.prev
                    next_node = current.next
                    prev.next = next_node
                    next_node.prev = prev
                    self.tasks.size -= 1
                
                print(f"Removed task: {task}")
                return True
            
            current = current.next
            if current == start:
                break
        
        print(f"Task ID {task_id} not found")
        return False
    
    def execute_round(self):
        """Execute one round of scheduling"""
        if self.tasks.head is None:
            print("No tasks to execute")
            return False
        
        current_task_node = self.tasks.head
        task = current_task_node.data
        
        print(f"\nTime {self.current_time}: Executing {task.name}")
        
        # Execute task
        executed_time, completed = task.execute(self.time_quantum)
        
        # Update statistics for waiting tasks
        self._update_waiting_times(task)
        
        self.current_time += executed_time
        
        if completed:
            task.turnaround_time = self.current_time
            self.completed_tasks.append(task)
            print(f"  ✓ Task {task.name} completed!")
            
            # Remove from scheduler
            self.remove_task(task.task_id)
            
            # If no more tasks
            if self.tasks.head is None:
                return False
        else:
            print(f"  → Task {task.name} needs {task.remaining_time} more time")
            
            # Move to next task (circular list automatically does this)
            self.tasks.head = self.tasks.head.next
        
        return True
    
    def _update_waiting_times(self, current_task):
        """Update waiting times for all tasks except current"""
        if self.tasks.head is None:
            return
        
        current = self.tasks.head
        start = current
        
        while True:
            if current.data != current_task:
                current.data.waiting_time += self.time_quantum
            
            current = current.next
            if current == start:
                break
    
    def run_scheduler(self):
        """Run scheduler until all tasks are complete"""
        print(f"\n{'='*60}")
        print("ROUND ROBIN SCHEDULER STARTING".center(60))
        print(f"{'='*60}")
        
        round_count = 1
        
        while self.execute_round():
            round_count += 1
            if round_count > 50:  # Safety limit
                print("Safety limit reached!")
                break
        
        self._print_statistics()
    
    def _print_statistics(self):
        """Print scheduling statistics"""
        print(f"\n{'='*60}")
        print("SCHEDULING COMPLETE".center(60))
        print(f"{'='*60}")
        
        print("\nCompleted Tasks:")
        print("-" * 60)
        print(f"{'Task':15} {'Burst Time':12} {'Waiting Time':14} {'Turnaround Time':16}")
        print("-" * 60)
        
        total_waiting = 0
        total_turnaround = 0
        
        for task in self.completed_tasks:
            print(f"{task.name:15} {task.burst_time:12} {task.waiting_time:14} {task.turnaround_time:16}")
            total_waiting += task.waiting_time
            total_turnaround += task.turnaround_time
        
        print("-" * 60)
        
        if self.completed_tasks:
            avg_waiting = total_waiting / len(self.completed_tasks)
            avg_turnaround = total_turnaround / len(self.completed_tasks)
            
            print(f"\nAverage Waiting Time: {avg_waiting:.2f}")
            print(f"Average Turnaround Time: {avg_turnaround:.2f}")
            print(f"Total Time: {self.current_time}")
    
    def display_queue(self):
        """Display current task queue"""
        if self.tasks.head is None:
            print("Task queue is empty")
            return
        
        print("\nCurrent Task Queue:")
        print("-" * 40)
        
        current = self.tasks.head
        start = current
        position = 1
        
        while True:
            task = current.data
            marker = "→ " if current == self.tasks.head else "  "
            status = f"[{task.remaining_time} remaining]"
            print(f"{marker}{position}. {task.name:15} {status}")
            
            current = current.next
            position += 1
            
            if current == start:
                break


class MultiLevelQueueScheduler:
    """Advanced: Multiple queues with different priorities"""
    def __init__(self):
        # High priority (System tasks) - Round Robin
        self.high_priority = RoundRobinScheduler(time_quantum=2)
        
        # Medium priority (Interactive tasks) - Round Robin
        self.medium_priority = RoundRobinScheduler(time_quantum=4)
        
        # Low priority (Batch tasks) - FCFS
        self.low_priority = []  # Simple list for FCFS
    
    def add_task(self, name, burst_time, priority):
        """Add task to appropriate queue"""
        if priority == 1:  # High
            self.high_priority.add_task(name, burst_time, priority)
        elif priority == 2:  # Medium
            self.medium_priority.add_task(name, burst_time, priority)
        else:  # Low
            task = Task(len(self.low_priority) + 1, name, burst_time, priority)
            self.low_priority.append(task)
            print(f"Added to low priority: {task}")
    
    def run(self):
        """Run multi-level scheduler"""
        print("\n" + "="*60)
        print("MULTI-LEVEL QUEUE SCHEDULER".center(60))
        print("="*60)
        
        # Execute high priority first
        if self.high_priority.tasks.head:
            print("\n>>> Executing HIGH Priority Tasks <<<")
            self.high_priority.run_scheduler()
        
        # Then medium priority
        if self.medium_priority.tasks.head:
            print("\n>>> Executing MEDIUM Priority Tasks <<<")
            self.medium_priority.run_scheduler()
        
        # Finally low priority (FCFS)
        if self.low_priority:
            print("\n>>> Executing LOW Priority Tasks (FCFS) <<<")
            self._run_fcfs()
    
    def _run_fcfs(self):
        """First Come First Served scheduler for low priority"""
        current_time = 0
        
        for task in self.low_priority:
            print(f"\nTime {current_time}: Executing {task.name}")
            task.waiting_time = current_time
            current_time += task.burst_time
            task.turnaround_time = current_time
            print(f"  ✓ Task {task.name} completed!")
        
        # Print statistics
        total_waiting = sum(t.waiting_time for t in self.low_priority)
        total_turnaround = sum(t.turnaround_time for t in self.low_priority)
        
        if self.low_priority:
            avg_waiting = total_waiting / len(self.low_priority)
            avg_turnaround = total_turnaround / len(self.low_priority)
            
            print(f"\nFCFS Statistics:")
            print(f"Average Waiting Time: {avg_waiting:.2f}")
            print(f"Average Turnaround Time: {avg_turnaround:.2f}")


# Example usage
def scheduler_demo():
    # Simple Round Robin Scheduler
    print("DEMO 1: ROUND ROBIN SCHEDULER")
    print("-" * 40)
    
    scheduler = RoundRobinScheduler(time_quantum=3)
    
    # Add some tasks
    scheduler.add_task("Web Server", 10, 1)
    scheduler.add_task("Database", 8, 2)
    scheduler.add_task("Backup", 5, 3)
    scheduler.add_task("Log Analysis", 7, 2)
    scheduler.add_task("Email Service", 6, 1)
    
    # Display initial queue
    scheduler.display_queue()
    
    # Run a few rounds manually
    print("\nExecuting first 3 rounds:")
    for i in range(3):
        scheduler.execute_round()
        scheduler.display_queue()
    
    # Or run complete scheduler
    # scheduler.run_scheduler()
    
    # Multi-Level Queue Scheduler
    print("\n\nDEMO 2: MULTI-LEVEL QUEUE SCHEDULER")
    print("-" * 40)
    
    ml_scheduler = MultiLevelQueueScheduler()
    
    # Add tasks with different priorities
    ml_scheduler.add_task("System Monitor", 4, 1)    # High
    ml_scheduler.add_task("User Interface", 8, 2)    # Medium
    ml_scheduler.add_task("Batch Processing", 12, 3) # Low
    ml_scheduler.add_task("Network Service", 5, 1)   # High
    ml_scheduler.add_task("File Download", 6, 2)     # Medium
    ml_scheduler.add_task("Data Backup", 15, 3)      # Low
    
    # Run the multi-level scheduler
    ml_scheduler.run()


if __name__ == "__main__":
    scheduler_demo()

# Other Real-World Applications

In [ ]:
class RingBuffer:
    """Fixed-size circular buffer using circular linked list"""
    def __init__(self, capacity):
        self.capacity = capacity
        self.buffer = SinglyCircularLinkedList()
        self.is_full = False
        
        # Initialize buffer with None values
        for _ in range(capacity):
            self.buffer.insert_at_end(None)
    
    def write(self, data):
        """Write data to buffer, overwriting old if full"""
        if self.buffer.head is None:
            return
        
        # Find next write position (first None or overwrite oldest)
        current = self.buffer.head
        
        while True:
            if current.data is None:
                current.data = data
                return
            
            current = current.next
            if current == self.buffer.head:
                # Buffer is full, overwrite head (oldest)
                self.buffer.head.data = data
                self.buffer.head = self.buffer.head.next
                self.is_full = True
                return
    
    def read(self):
        """Read oldest data"""
        if self.buffer.head is None or (not self.is_full and self.buffer.head.data is None):
            return None
        
        data = self.buffer.head.data
        self.buffer.head.data = None
        self.buffer.head = self.buffer.head.next
        self.is_full = False
        return data

# Application 2: Multiplayer Game Turn System

In [ ]:
class Player:
    def __init__(self, name, character_class):
        self.name = name
        self.character_class = character_class
        self.health = 100
        self.mana = 50
    
    def __str__(self):
        return f"{self.name} ({self.character_class}) - HP: {self.health}, MP: {self.mana}"

class GameTurnSystem:
    def __init__(self):
        self.players = SinglyCircularLinkedList()
        self.current_player = None
    
    def add_player(self, name, character_class):
        player = Player(name, character_class)
        self.players.insert_at_end(player)
        
        if self.current_player is None:
            self.current_player = self.players.head
        
        print(f"Player added: {player}")
    
    def next_turn(self):
        if self.current_player is None:
            return None
        
        current = self.current_player.data
        print(f"\n=== {current.name}'s turn ===")
        
        # Move to next player
        self.current_player = self.current_player.next
        
        return current

1. Split Circular List into Two Halves

In [ ]:
def split_circular_list(clist):
    """Split circular list into two equal halves"""
    if clist.head is None or clist.size < 2:
        return clist, None
    
    # Use slow-fast pointer to find middle
    slow = clist.head
    fast = clist.head
    
    while fast.next != clist.head and fast.next.next != clist.head:
        slow = slow.next
        fast = fast.next.next
    
    # Create second half
    second_half = SinglyCircularLinkedList()
    
    if fast.next.next == clist.head:
        fast = fast.next
    
    # Split
    second_half.head = slow.next
    fast.next = second_half.head
    
    # Make first half circular
    slow.next = clist.head
    
    # Update sizes
    clist.size = clist.size // 2 + clist.size % 2
    second_half.size = clist.size // 2
    
    return clist, second_half

2. Check if Linked List is Circular

In [ ]:
def is_circular(linked_list):
    """Check if a linked list is circular"""
    if linked_list.head is None:
        return True  # Empty list is trivially circular
    
    slow = linked_list.head
    fast = linked_list.head
    
    while fast and fast.next:
        slow = slow.next
        fast = fast.next.next
        
        if slow == fast:
            # We have a cycle, now check if it's circular (back to head)
            slow = linked_list.head
            while True:
                if slow == fast:
                    return True
                slow = slow.next
                fast = fast.next
    
    return False